In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST',one_hot=True)

Extracting MNIST\train-images-idx3-ubyte.gz
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz


## weight and bias

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

## conv2d and max_pool

In [7]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding ='SAME')

##  conpute accuracy

In [35]:
def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={# if it is not neccessary ???
            xs:v_xs,
            ys:v_ys,
            keep_prob:0.5
        })
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict ={
            xs:v_xs,
            ys:v_ys,
            keep_prob:0.5
        })
    return result

## Modeling

### define placeholder and transform the tensor

In [10]:
xs = tf.placeholder(tf.float32,[None,784])
ys = tf.placeholder(tf.float32,[None,10])

x_image = tf.reshape(xs,[-1,28,28,1])
keep_prob = tf.placeholder(tf.float32)

###  Conv1 layer and max pooling

In [11]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

In [14]:
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Conv2 layer

In [15]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

In [16]:
h_conv2 = conv2d(h_pool1,W_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### full conect1

In [17]:
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])

In [21]:
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

### full conect2

In [22]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

In [23]:
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

### define loss and optimizer

In [29]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(-ys*tf.log(prediction),reduction_indices = [1]))

In [30]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## Train-------------------------

### Initial   a session

In [31]:
sess = tf.Session()

In [32]:
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(1000):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    sess.run(train_step,feed_dict={
            xs:batch_xs,
            ys:batch_ys,
            keep_prob:0.5
        })
    
    if i% 50 ==0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))